# Candle Dataset Cleaning

After running the first training job on the full dataset, we can see that despite the training loss decreasing, the validation loss is remaining fairly constant. As a potential remedy, we are going to try removing all of the rows from a closed market from the dataset. This will have a few advantages:
1. It will drasitcally reduce the size of the data by approximately 55 - 60%, by keeping only the examples that are relevant to the task we need to perform. 
2. It will ensure that the loss metrics are mearusing the models performnace on market data where trading is occuring. Currently, the loss function is likely dilluted, as so much of the data is just a flat time-series, that the model is likely performing perfectly on these examples, causing the loss metric to be diluted, and likely not allowing a high enough gradient to be built up for the backward pass.

## Plan for Cleaning Data 

1. Remove any rows where the market is closed. We will never be using this in the actual model.
2. This should give us a a couple hundred time-series per ticker. We want to create a time-series ID for each continuous time series in the dataset. This will be used as a column ID.
3. Then iterate over each ticker. For each ticker:
 - Use our date indicices, to create a train, validation, and test set for that ticker. Be sure that the date the sets do not slice any time-series (e.g. each of the 3 sets should have a unique set of time-series IDs)
 - Append eacah of the three sets to a master train, validation, and test set respectively. 
 - These steps will replace the current `select_by_index` usage
4. Train the `TimeSeriesPreprocessor` on the train set
5. Create the train, validation, and test datasets, using the trained preprocessor



In [44]:
# Standard
import os
import random

# Third Party
from transformers import (
    EarlyStoppingCallback,
    PatchTSMixerConfig,
    PatchTSMixerForPrediction,
    Trainer,
    TrainingArguments,
)
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

# First Party
from tsfm_public.toolkit.dataset import ForecastDFDataset
from tsfm_public.toolkit.time_series_preprocessor import TimeSeriesPreprocessor


In [45]:
# Set seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

## Dataset Preprocessing

We want to meticulously craft our examples for each trading day. Our primary focus here is to ensure that the model only is ever asked to forecast into live market data. We need it to understand this type of forecasting, and not be thrown off at all by examples from extended hours trading.

There are a few general steps we must take:
1. Ensure that all the data from the closed market is removed from the dataset, as this data will never be used in context or in forecasting.
2. Ensure that the timestamps are localized to America/New_York, so that we can get accurate date_strings to use for identifying each trading day.

In [103]:
# Load the Dataset from the CSV file
DATA_DIR = "/home/ubuntu/verb-workspace/data" # set this accordingly to the location of the data

MORNING_TRAIN_DATASET = f"{DATA_DIR}/1min-candles-train-MORNING.csv"
MORNING_VALID_DATASET = f"{DATA_DIR}/1min-candles-valid-MORNING.csv"
MORNING_TEST_DATASET = f"{DATA_DIR}/1min-candles-test-MORNING.csv"

DAY_TRAIN_DATASET = f"{DATA_DIR}/1min-candles-train-DAY.csv"
DAY_VALID_DATASET = f"{DATA_DIR}/1min-candles-valid-DAY.csv"
DAY_TEST_DATASET = f"{DATA_DIR}/1min-candles-test-DAY.csv"

AFTERNOON_TRAIN_DATASET = f"{DATA_DIR}/1min-candles-train-AFTERNOON.csv"
AFTERNOON_VALID_DATASET = f"{DATA_DIR}/1min-candles-valid-AFTERNOON.csv"
AFTERNOON_TEST_DATASET = f"{DATA_DIR}/1min-candles-test-AFTERNOON.csv"


timestamp_col = 't'

morning_train_data = pd.read_csv(
    MORNING_TRAIN_DATASET,
    parse_dates=[timestamp_col]
)
print("Morning Train Loaded")

morning_valid_data = pd.read_csv(
    MORNING_VALID_DATASET,
    parse_dates=[timestamp_col]
)
print("Morning Valid Loaded")


morning_test_data = pd.read_csv(
    MORNING_TEST_DATASET,
    parse_dates=[timestamp_col]
)
print("Morning Test Loaded")



day_train_data = pd.read_csv(
    DAY_TRAIN_DATASET,
    parse_dates=[timestamp_col]
)
print("Day Train Loaded")

day_valid_data = pd.read_csv(
    DAY_VALID_DATASET,
    parse_dates=[timestamp_col]
)
print("Day Valid Loaded")

day_test_data = pd.read_csv(
    DAY_TEST_DATASET,
    parse_dates=[timestamp_col]
)
print("Day Test Loaded")


afternoon_train_data = pd.read_csv(
    AFTERNOON_TRAIN_DATASET,
    parse_dates=[timestamp_col]
)
print("Afternoon Train Loaded")

afternoon_valid_data = pd.read_csv(
    AFTERNOON_VALID_DATASET,
    parse_dates=[timestamp_col]
)
print("Afternoon Valid Loaded")


afternoon_test_data = pd.read_csv(
    AFTERNOON_TEST_DATASET,
    parse_dates=[timestamp_col]
)
print("Afternoon Test Loaded")


Morning Train Loaded
Morning Valid Loaded
Morning Test Loaded
Day Train Loaded
Day Valid Loaded
Day Test Loaded
Afternoon Train Loaded
Afternoon Valid Loaded
Afternoon Test Loaded


In [128]:
from tqdm import tqdm
from pandarallel import pandarallel

pandarallel.initialize(nb_workers=12)

desired_order = ['ticker', 'date_string', 't', 'targ_o', 'targ_h', 'targ_l', 'targ_c', 'targ_v', 'targ_vwap', 'targ_red', 'targ_green', 'cont_market_open', 'cont_market_extended']

# Define custom aggregation functions
def vwap_agg(x):
    if isinstance(x, pd.Series):
        return 0
    return (x.targ_v * x.targ_c).sum() / x.targ_v.sum() if x.targ_v.sum() > 0 else 0

agg_dict = {
    'targ_o': 'first',
    'targ_h': 'max',
    'targ_l': 'min',
    'targ_c': 'last',
    'targ_v': 'sum',
    'targ_vwap': vwap_agg,
    'cont_market_open': 'max',
    'cont_market_extended': 'max'
}

def custom_agg(x):
    agg_values = {
        'targ_o': x.targ_o.iloc[0],
        'targ_h': x.targ_h.max(),
        'targ_l': x.targ_l.min(),
        'targ_c': x.targ_c.iloc[-1],
        'targ_v': x.targ_v.sum(),
        'targ_vwap': vwap_agg(x),
        'cont_market_open': x.cont_market_open.max(),
        'cont_market_extended': x.cont_market_extended.max()
    }
    return pd.Series(agg_values)

def aggregate_min_candles(df: pd.DataFrame) -> pd.DataFrame:
    # Set 'date' as the index of the dataframe
    # Be sure to make a copy here, so as to not mutate the original
    _df = df.set_index(timestamp_col)
    
    # Group by 'ticker' and 'date_string', and resample the data to 1-minute frequency
    groups = _df.groupby(['ticker', 'date_string'])
    
    def process_group(group):
        ticker = group.iloc[0].ticker
        date_str = group.iloc[0].date_string
        group.index = pd.to_datetime(group.index)
        resampled_group = group.resample('3min').apply(custom_agg)

        # Add the targ_red and targ_green columns back in
        resampled_group['targ_red'] = (resampled_group['targ_c'] < resampled_group['targ_o']).astype(int)
        resampled_group['targ_green'] = (resampled_group['targ_c'] > resampled_group['targ_o']).astype(int)
        resampled_group = resampled_group.reset_index()
        resampled_group['ticker'] = ticker
        resampled_group['date_string'] = date_str
        resampled_group = resampled_group.reindex(columns=desired_order)

        return resampled_group
    
    resampled_groups = groups.parallel_apply(process_group)
    resampled_df = resampled_groups.reset_index(drop=True)
    
    
    return resampled_df
    
datasets = {
    "morning_train": morning_train_data,
    "morning_valid": morning_valid_data,
    "morning_test": morning_test_data,
    "day_train": day_train_data,
    "day_valid": day_valid_data,
    "day_test": day_test_data,
    "afternoon_train": afternoon_train_data,
    "afternoon_valid": afternoon_valid_data,
    "afternoon_test": afternoon_test_data,
}

resampled_datasets = {}
for key, dataset in tqdm(datasets.items(), total=len(datasets)):
    resampled_dataset = aggregate_min_candles(dataset)
    resampled_datasets[key] = resampled_dataset



INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


100%|██████████| 9/9 [02:31<00:00, 16.83s/it]


In [129]:
morning_train_3min = resampled_datasets['morning_train']
morning_valid_3min = resampled_datasets['morning_valid']
morning_test_3min = resampled_datasets['morning_test']

day_train_3min = resampled_datasets['day_train']
day_valid_3min = resampled_datasets['day_valid']
day_test_3min = resampled_datasets['day_test']

afternoon_train_3min = resampled_datasets['afternoon_train']
afternoon_valid_3min = resampled_datasets['afternoon_valid']
afternoon_test_3min = resampled_datasets['afternoon_test']

In [130]:
# # Let's look at the first examples from the morning_train set
example_day = morning_train_3min.loc[(morning_train_3min.ticker == 'AAPL') & (morning_train_3min.date_string == '2023-01-03')]
example_day


,ticker,date_string,t,targ_o,targ_h,targ_l,targ_c,targ_v,targ_vwap,targ_red,targ_green,cont_market_open,cont_market_extended
0,AAPL,2023-01-03,2023-01-03 05:30:00-05:00,130.8000,130.8000,130.800,130.8000,0.0,0.000000,0,0,0.0,1.0
1,AAPL,2023-01-03,2023-01-03 05:33:00-05:00,130.8000,130.8000,130.800,130.8000,235.0,130.800000,0,0,0.0,1.0
2,AAPL,2023-01-03,2023-01-03 05:36:00-05:00,130.8000,130.8000,130.800,130.8000,0.0,0.000000,0,0,0.0,1.0
3,AAPL,2023-01-03,2023-01-03 05:39:00-05:00,130.8000,130.8000,130.800,130.8000,0.0,0.000000,0,0,0.0,1.0
4,AAPL,2023-01-03,2023-01-03 05:42:00-05:00,130.8000,131.0200,130.800,131.0000,4873.0,130.972664,0,1,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,AAPL,2023-01-03,2023-01-03 10:48:00-05:00,125.4509,125.7250,125.400,125.4745,753140.0,125.581425,0,1,1.0,0.0
107,AAPL,2023-01-03,2023-01-03 10:51:00-05:00,125.4800,125.5400,125.230,125.3750,1100388.0,125.352022,1,0,1.0,0.0
108,AAPL,2023-01-03,2023-01-03 10:54:00-05:00,125.3800,125.5100,125.290,125.4100,513210.0,125.386915,0,1,1.0,0.0
109,AAPL,2023-01-03,2023-01-03 10:57:00-05:00,125.4100,125.6558,125.405,125.5950,607672.0,125.538300,0,1,1.0,0.0


We can check the 111 is the right number of rows:

11:00 - 05:30 = 5 hours and 30 minutes = 330 minutes / 3 = 111

In [131]:
# Now let's assert that every morning group has this correct row count
train_groups = morning_train_3min.groupby(['ticker', 'date_string'])
valid_groups = morning_valid_3min.groupby(['ticker', 'date_string'])
test_groups = morning_test_3min.groupby(['ticker', 'date_string'])

NUM_ROWS = 111

# Create a list of group objects
all_groups = [train_groups, valid_groups, test_groups]

for groups in all_groups:
    for (ticker, date_string), group in groups:
        assert len(group) == NUM_ROWS

In [132]:
# Let's look at the first examples from the day_train set
example_day = day_train_3min.loc[(day_train_3min.ticker == 'AAPL') & (day_train_3min.date_string == '2023-01-03')]
example_day

,ticker,date_string,t,targ_o,targ_h,targ_l,targ_c,targ_v,targ_vwap,targ_red,targ_green,cont_market_open,cont_market_extended
0,AAPL,2023-01-03,2023-01-03 07:00:00-05:00,130.3800,130.450,130.03,130.080,27926.0,130.138507,1,0,0.0,1.0
1,AAPL,2023-01-03,2023-01-03 07:03:00-05:00,130.0800,130.250,130.08,130.250,12442.0,130.222328,0,1,0.0,1.0
2,AAPL,2023-01-03,2023-01-03 07:06:00-05:00,130.2800,130.350,130.25,130.300,8827.0,130.268710,0,1,0.0,1.0
3,AAPL,2023-01-03,2023-01-03 07:09:00-05:00,130.2500,130.300,130.20,130.250,1963.0,130.229343,0,0,0.0,1.0
4,AAPL,2023-01-03,2023-01-03 07:12:00-05:00,130.2200,130.380,130.22,130.380,11643.0,130.341838,0,1,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,AAPL,2023-01-03,2023-01-03 14:18:00-05:00,124.7100,124.720,124.48,124.510,419116.0,124.551502,1,0,1.0,0.0
147,AAPL,2023-01-03,2023-01-03 14:21:00-05:00,124.5097,124.580,124.43,124.470,314932.0,124.497117,1,0,1.0,0.0
148,AAPL,2023-01-03,2023-01-03 14:24:00-05:00,124.4750,124.560,124.42,124.545,402908.0,124.518821,0,1,1.0,0.0
149,AAPL,2023-01-03,2023-01-03 14:27:00-05:00,124.5483,124.609,124.46,124.570,318407.0,124.538265,0,1,1.0,0.0


We can check the 111 is the right number of rows:

14:33 - 07:00 = 7 hours and 33 minutes = 453 minutes / 3 = 151

In [133]:
# Now let's assert that every day group has this correct row count
train_groups = day_train_3min.groupby(['ticker', 'date_string'])
valid_groups = day_valid_3min.groupby(['ticker', 'date_string'])
test_groups = day_test_3min.groupby(['ticker', 'date_string'])

NUM_ROWS = 151

# Create a list of group objects
all_groups = [train_groups, valid_groups, test_groups]

for groups in all_groups:
    for (ticker, date_string), group in groups:
        assert len(group) == NUM_ROWS

In [134]:
# Let's look at the first examples from the afternoon_train set
example_day = afternoon_train_3min.loc[(afternoon_train_3min.ticker == 'AAPL') & (afternoon_train_3min.date_string == '2023-01-03')]
example_day

,ticker,date_string,t,targ_o,targ_h,targ_l,targ_c,targ_v,targ_vwap,targ_red,targ_green,cont_market_open,cont_market_extended
0,AAPL,2023-01-03,2023-01-03 10:30:00-05:00,125.460,125.6900,125.100,125.392,1297207.0,125.325873,1,0,1.0,0.0
1,AAPL,2023-01-03,2023-01-03 10:33:00-05:00,125.390,125.3900,125.120,125.130,1179193.0,125.165956,1,0,1.0,0.0
2,AAPL,2023-01-03,2023-01-03 10:36:00-05:00,125.125,125.3900,125.050,125.360,1055173.0,125.200225,0,1,1.0,0.0
3,AAPL,2023-01-03,2023-01-03 10:39:00-05:00,125.365,125.7800,125.300,125.770,1159752.0,125.613044,0,1,1.0,0.0
4,AAPL,2023-01-03,2023-01-03 10:42:00-05:00,125.760,125.7671,125.310,125.350,764586.0,125.515580,1,0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,AAPL,2023-01-03,2023-01-03 15:45:00-05:00,124.990,125.0000,124.885,124.990,601944.0,124.951968,0,0,1.0,0.0
106,AAPL,2023-01-03,2023-01-03 15:48:00-05:00,124.980,125.2000,124.810,125.100,1711512.0,125.067577,0,1,1.0,0.0
107,AAPL,2023-01-03,2023-01-03 15:51:00-05:00,125.095,125.1900,125.030,125.140,1039197.0,125.127620,0,1,1.0,0.0
108,AAPL,2023-01-03,2023-01-03 15:54:00-05:00,125.140,125.4200,125.040,125.150,2002053.0,125.225498,0,1,1.0,0.0


We can check the 111 is the right number of rows:

15:57 - 10:30 = 5 hours and 27 minutes = 327 minutes / 3 = 110

In [135]:
# Now let's assert that every afternoon group has this correct row count
train_groups = afternoon_train_3min.groupby(['ticker', 'date_string'])
valid_groups = afternoon_valid_3min.groupby(['ticker', 'date_string'])
test_groups = afternoon_test_3min.groupby(['ticker', 'date_string'])

NUM_ROWS = 110

# Create a list of group objects
all_groups = [train_groups, valid_groups, test_groups]

for groups in all_groups:
    for (ticker, date_string), group in groups:
        assert len(group) == NUM_ROWS

In [136]:
MORNING_TRAIN_DATASET_3MIN = f"{DATA_DIR}/3min-candles-train-MORNING.csv"
MORNING_VALID_DATASET_3MIN = f"{DATA_DIR}/3min-candles-valid-MORNING.csv"
MORNING_TEST_DATASET_3MIN = f"{DATA_DIR}/3min-candles-test-MORNING.csv"

DAY_TRAIN_DATASET_3MIN = f"{DATA_DIR}/3min-candles-train-DAY.csv"
DAY_VALID_DATASET_3MIN = f"{DATA_DIR}/3min-candles-valid-DAY.csv"
DAY_TEST_DATASET_3MIN = f"{DATA_DIR}/3min-candles-test-DAY.csv"

AFTERNOON_TRAIN_DATASET_3MIN = f"{DATA_DIR}/3min-candles-train-AFTERNOON.csv"
AFTERNOON_VALID_DATASET_3MIN = f"{DATA_DIR}/3min-candles-valid-AFTERNOON.csv"
AFTERNOON_TEST_DATASET_3MIN = f"{DATA_DIR}/3min-candles-test-AFTERNOON.csv"

morning_train_3min.to_csv(MORNING_TRAIN_DATASET_3MIN, index=False)
morning_valid_3min.to_csv(MORNING_VALID_DATASET_3MIN, index=False)
morning_test_3min.to_csv(MORNING_TEST_DATASET_3MIN, index=False)

day_train_3min.to_csv(DAY_TRAIN_DATASET_3MIN, index=False)
day_valid_3min.to_csv(DAY_VALID_DATASET_3MIN, index=False)
day_test_3min.to_csv(DAY_TEST_DATASET_3MIN, index=False)

afternoon_train_3min.to_csv(AFTERNOON_TRAIN_DATASET_3MIN, index=False)
afternoon_valid_3min.to_csv(AFTERNOON_VALID_DATASET_3MIN, index=False)
afternoon_test_3min.to_csv(AFTERNOON_TEST_DATASET_3MIN, index=False)